# 🚀 GitHub-Sync Master Control

**Panneau de contrôle centralisé pour système GitHub-Sync**

- Configuration stratégies de mise à jour
- Orchestration cycles complets
- Monitoring et rollback
- Interface commande unifiée

---

In [ ]:
# 🔧 INITIALISATION MASTER CONTROL
import sys
import os
from datetime import datetime

# Déterminer le chemin du repository
repo_path = '/content/PaniniFS-Research'
github_sync_path = os.path.join(repo_path, 'src')

# Ajouter le chemin src au Python path
if github_sync_path not in sys.path:
    sys.path.insert(0, github_sync_path)

# Import modules GitHub-Sync
try:
    from github_sync.config_manager import GitHubSyncConfig
    from github_sync.module_updater import ModuleUpdater
    from github_sync.github_loader import GitHubModuleLoader
    from github_sync.hot_reload import HotReloadManager
    
    print("✅ Modules GitHub-Sync importés avec succès")
    
except ImportError as e:
    print(f"⚠️ Modules GitHub-Sync non disponibles: {e}")
    print("🔄 Clonage repository en cours...")
    
    # Cloner repository si nécessaire
    if not os.path.exists(repo_path):
        !git clone https://github.com/stephanedenis/PaniniFS-Research.git {repo_path}
    
    # Ajouter le chemin src au Python path
    if github_sync_path not in sys.path:
        sys.path.insert(0, github_sync_path)
    
    # Réimporter
    from github_sync.config_manager import GitHubSyncConfig
    from github_sync.module_updater import ModuleUpdater
    from github_sync.github_loader import GitHubModuleLoader
    from github_sync.hot_reload import HotReloadManager
    
    print("✅ Modules GitHub-Sync installés et importés")

print(f"🕒 Master Control initialisé: {datetime.now().strftime('%H:%M:%S')}")
print(f"📁 Repository path: {repo_path}")
print(f"🐍 Python path ajouté: {github_sync_path}")

In [ ]:
# 🎛️ PANNEAU CONFIGURATION

# Initialiser gestionnaire configuration
config = GitHubSyncConfig()

print("🔧 CONFIGURATION GITHUB-SYNC")
print("=" * 50)

# Afficher configuration actuelle
summary = config.get_config_summary()
print(f"📊 Résumé Configuration:")
print(f"├── Stratégie: {summary['strategy'].upper()}")
print(f"├── Auto-rollback: {'✅' if summary['auto_rollback'] else '❌'}")
print(f"├── Intervalle vérif: {summary['check_interval_min']} min")
print(f"├── Modules cibles: {summary['target_modules_count']}")
print(f"├── Hot-reload: {'✅' if summary['hot_reload_enabled'] else '❌'}")
print(f"└── Safety checks: {'✅' if summary['safety_checks'] else '❌'}")

print("\n🎯 Modules Cibles:")
target_modules = config.get_module_paths()
for i, module in enumerate(target_modules, 1):
    print(f"{i:2d}. {module}")

# Validation configuration
validation = config.validate_config()
print(f"\n🔍 Validation: {'✅ VALIDE' if validation['valid'] else '❌ INVALIDE'}")

if validation['warnings']:
    print("⚠️ Avertissements:")
    for warning in validation['warnings']:
        print(f"  • {warning}")

if validation['errors']:
    print("❌ Erreurs:")
    for error in validation['errors']:
        print(f"  • {error}")

print("\n" + "=" * 50)

In [ ]:
# 🚀 ORCHESTRATEUR PRINCIPAL

# Initialiser orchestrateur
updater = ModuleUpdater()

print("🚀 ORCHESTRATEUR GITHUB-SYNC")
print("=" * 50)

# Configuration stratégie selon besoin
STRATEGY = "conservative"  # conservative, aggressive, manual

print(f"🔧 Configuration stratégie: {STRATEGY}")
updater.configure_update_strategy(STRATEGY)

# État actuel système
print("\n📊 État Système:")
status = updater.get_update_status()
print(f"├── Stratégie active: {status['strategy']}")
print(f"├── Auto-rollback: {'✅' if status['auto_rollback'] else '❌'}")
print(f"├── Dernière MAJ: {status['last_update'] or 'Jamais'}")
print(f"├── Historique MAJ: {status['update_count']} cycles")
print(f"└── Points rollback: {status['rollback_points']}")

# Points de rollback disponibles
rollback_points = updater.list_available_rollback_points()
if rollback_points:
    print("\n⏰ Points Rollback Disponibles:")
    for i, point in enumerate(rollback_points, 1):
        age = point['age_hours']
        fallback = '✅' if point['fallback_available'] else '❌'
        print(f"{i:2d}. {point['timestamp'][:19]} (il y a {age:.1f}h) - Fallback: {fallback}")
else:
    print("\n⏰ Aucun point de rollback disponible")

print("\n" + "=" * 50)

In [ ]:
# 🔄 EXÉCUTION CYCLE COMPLET

print("🔄 EXÉCUTION CYCLE MISE À JOUR")
print("=" * 50)

# Options d'exécution
FORCE_UPDATE = False  # True pour forcer même sans changements
DRY_RUN = False       # True pour simulation sans modifications

if DRY_RUN:
    print("🧪 MODE SIMULATION (DRY RUN)")
    print("Aucune modification ne sera effectuée")
    print()

# Démarrage cycle
print(f"🚀 Démarrage cycle complet...")
print(f"├── Force update: {'✅' if FORCE_UPDATE else '❌'}")
print(f"├── Stratégie: {updater.update_strategy}")
print(f"└── Heure: {datetime.now().strftime('%H:%M:%S')}")
print()

if not DRY_RUN:
    # Exécution réelle
    cycle_result = updater.execute_full_update_cycle(force=FORCE_UPDATE)
    
    print("\n📋 RÉSULTAT CYCLE:")
    print(f"├── ID Cycle: {cycle_result['cycle_id']}")
    print(f"├── Succès: {'✅' if cycle_result['success'] else '❌'}")
    print(f"├── Durée: {cycle_result.get('total_duration', 0):.2f}s")
    print(f"├── Stratégie: {cycle_result['strategy']}")
    
    if 'reason' in cycle_result:
        print(f"├── Raison: {cycle_result['reason']}")
    
    if cycle_result.get('rollback_performed'):
        print(f"└── Rollback: ✅ Exécuté")
    else:
        print(f"└── Rollback: ❌ Non nécessaire")
    
    # Détail des phases
    if 'phases' in cycle_result:
        print("\n📊 Détail Phases:")
        for phase_name, phase_data in cycle_result['phases'].items():
            duration = phase_data.get('duration', 0)
            print(f"├── {phase_name}: {duration:.2f}s")
            
            # Détails spécifiques par phase
            if phase_name == 'download' and 'downloaded' in phase_data:
                downloaded = len(phase_data['downloaded'])
                failed = len(phase_data['failed'])
                print(f"│   ├── Téléchargés: {downloaded}")
                print(f"│   └── Échecs: {failed}")
            
            elif phase_name == 'validation' and 'valid' in phase_data:
                valid = len(phase_data['valid'])
                invalid = len(phase_data['invalid'])
                print(f"│   ├── Valides: {valid}")
                print(f"│   └── Invalides: {invalid}")
    
    # État système post-cycle
    print("\n📊 État Post-Cycle:")
    post_status = updater.get_update_status()
    print(f"├── Dernière MAJ: {post_status['last_update'] or 'Aucune'}")
    print(f"├── Total cycles: {post_status['update_count']}")
    print(f"└── Points rollback: {post_status['rollback_points']}")

else:
    # Mode simulation
    print("🧪 SIMULATION - Actions qui seraient exécutées:")
    print("1. 📡 Vérification versions GitHub")
    print("2. 📥 Téléchargement modules mis à jour")
    print("3. 🧪 Validation modules téléchargés")
    print("4. 💾 Création point rollback")
    print("5. 🔄 Hot-reload modules")
    print("6. 🧪 Tests post-reload")
    print("7. 📊 Mise à jour état système")

print("\n" + "=" * 50)
print(f"🏁 Cycle terminé: {datetime.now().strftime('%H:%M:%S')}")

In [ ]:
# 🎮 PANNEAU CONTRÔLE INTERACTIF

print("🎮 PANNEAU CONTRÔLE INTERACTIF")
print("=" * 50)

def show_control_menu():
    """Affiche le menu de contrôle"""
    print("\n🎯 Actions Disponibles:")
    print("1. 🔄 Déclencher mise à jour manuelle")
    print("2. ⚡ Mise à jour forcée (ignore cache)")
    print("3. ↩️  Rollback vers point précédent")
    print("4. 🔧 Changer stratégie")
    print("5. 📊 Afficher état détaillé")
    print("6. 📁 Gérer modules cibles")
    print("7. 💾 Sauvegarder configuration")
    print("8. 🔄 Recharger configuration")
    print("0. 🏁 Quitter")

def execute_action(choice):
    """Exécute l'action choisie"""
    
    if choice == "1":
        print("🔄 Déclenchement mise à jour manuelle...")
        result = updater.manual_update_trigger()
        print(f"Résultat: {'✅ Succès' if result['success'] else '❌ Échec'}")
        
    elif choice == "2":
        print("⚡ Mise à jour forcée...")
        result = updater.manual_update_trigger(force=True)
        print(f"Résultat: {'✅ Succès' if result['success'] else '❌ Échec'}")
        
    elif choice == "3":
        rollback_points = updater.list_available_rollback_points()
        if rollback_points:
            print("↩️ Rollback vers dernier point...")
            success = updater.manual_rollback()
            print(f"Rollback: {'✅ Réussi' if success else '❌ Échoué'}")
        else:
            print("❌ Aucun point de rollback disponible")
            
    elif choice == "4":
        print("🔧 Changement stratégie:")
        print("  1. Conservative (sécurisé)")
        print("  2. Aggressive (rapide)")
        print("  3. Manual (contrôle total)")
        
        strategy_choice = input("Choisir stratégie (1-3): ").strip()
        strategies = {"1": "conservative", "2": "aggressive", "3": "manual"}
        
        if strategy_choice in strategies:
            new_strategy = strategies[strategy_choice]
            updater.configure_update_strategy(new_strategy)
            config.configure_strategy(new_strategy)
            print(f"✅ Stratégie changée: {new_strategy}")
        else:
            print("❌ Choix invalide")
            
    elif choice == "5":
        print("📊 État Détaillé:")
        status = updater.get_update_status()
        
        for key, value in status.items():
            if isinstance(value, dict):
                print(f"├── {key}:")
                for sub_key, sub_value in value.items():
                    print(f"│   ├── {sub_key}: {sub_value}")
            else:
                print(f"├── {key}: {value}")
                
    elif choice == "6":
        print("📁 Gestion Modules Cibles:")
        modules = config.get_module_paths()
        print(f"Modules actuels: {len(modules)}")
        for i, module in enumerate(modules, 1):
            print(f"{i:2d}. {module}")
            
    elif choice == "7":
        print("💾 Sauvegarde configuration...")
        success = config.save_config()
        print(f"Sauvegarde: {'✅ Réussie' if success else '❌ Échouée'}")
        
    elif choice == "8":
        print("🔄 Rechargement configuration...")
        success = config.load_config()
        print(f"Rechargement: {'✅ Réussi' if success else '❌ Échoué'}")
        
    elif choice == "0":
        print("🏁 Arrêt panneau contrôle")
        return False
        
    else:
        print("❌ Choix invalide")
    
    return True

# Menu interactif (décommenté pour utilisation)
show_control_menu()
print("\n💡 Pour utiliser le menu interactif, exécutez:")
print("choice = input('Action (0-8): ')")
print("execute_action(choice)")

print("\n" + "=" * 50)
print("🎮 Panneau contrôle prêt")